# 🚀 Vesuvius Challenge - Runpods完全独立版（実データ対応）

**このノートブック1つで完結** - 外部ファイル依存なし

## 📥 Step 1: Kaggle認証とデータダウンロード

In [ ]:
# Kaggle認証設定とデータダウンロード
import os
import sys
import json
import subprocess
import zipfile
from pathlib import Path

def setup_kaggle_auth(username=None, key=None):
    """Kaggle認証設定（Runpods対応）"""
    
    # 既存のkaggle.json確認
    locations = [
        Path("/workspace/kaggle.json"),
        Path("/workspace/.kaggle/kaggle.json"),
        Path.home() / ".kaggle" / "kaggle.json",
        Path("./kaggle.json"),
        Path("/content/kaggle.json")
    ]
    
    for loc in locations:
        if loc.exists():
            print(f"✅ Kaggle認証ファイル発見: {loc}")
            if loc.parent.name != ".kaggle":
                os.environ['KAGGLE_CONFIG_DIR'] = str(loc.parent)
            return True
    
    # 認証情報が提供された場合は作成
    if username and key:
        kaggle_config = {"username": username, "key": key}
        
        # Runpods環境チェック
        if Path("/workspace").exists():
            kaggle_path = Path("/workspace/kaggle.json")
        else:
            kaggle_path = Path("./kaggle.json")
        
        with open(kaggle_path, 'w') as f:
            json.dump(kaggle_config, f)
        
        os.chmod(kaggle_path, 0o600)
        os.environ['KAGGLE_CONFIG_DIR'] = str(kaggle_path.parent)
        print(f"✅ Kaggle認証設定完了: {kaggle_path}")
        return True
    
    print("⚠️ Kaggle認証が必要です")
    print("次のセルでsetup_kaggle_auth(username='YOUR_USERNAME', key='YOUR_KEY')を実行してください")
    return False

# 認証確認
auth_ok = setup_kaggle_auth()

# Kaggle APIインストール
try:
    import kaggle
    print("✅ Kaggle API準備完了")
except ImportError:
    print("📦 Kaggle APIインストール中...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kaggle", "-q"])
    print("✅ インストール完了")

In [ ]:
# 認証情報を設定する場合（必要に応じて実行）
# setup_kaggle_auth(username="YOUR_KAGGLE_USERNAME", key="YOUR_KAGGLE_API_KEY")

In [ ]:
# データダウンロード
def download_vesuvius_data():
    """Vesuviusデータセットをダウンロード"""
    
    # データディレクトリ設定
    if Path("/workspace").exists():
        data_dir = Path("/workspace/data")
    else:
        data_dir = Path("./data")
    
    data_dir.mkdir(parents=True, exist_ok=True)
    dataset_dir = data_dir / "vesuvius-challenge-surface-detection"
    
    # 既存データ確認
    if dataset_dir.exists():
        train_images = dataset_dir / "train_images"
        if train_images.exists():
            tiff_files = list(train_images.glob("*.tif"))
            if len(tiff_files) > 0:
                print(f"✅ データ既存: {len(tiff_files)}画像")
                return str(dataset_dir)
    
    print("📥 データダウンロード開始（時間がかかります）...")
    
    try:
        # Kaggle CLIでダウンロード
        cmd = f"kaggle competitions download -c vesuvius-challenge-surface-detection -p {data_dir}"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=600)
        
        if result.returncode == 0:
            print("✅ ダウンロード完了")
            
            # ZIP解凍
            zip_file = data_dir / "vesuvius-challenge-surface-detection.zip"
            if zip_file.exists():
                print("📦 解凍中...")
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall(data_dir)
                zip_file.unlink()
                print("✅ 解凍完了")
            
            return str(dataset_dir)
        else:
            print(f"⚠️ ダウンロードエラー: {result.stderr}")
            
    except subprocess.TimeoutExpired:
        print("⏱️ タイムアウト - バックグラウンドで継続中")
    except Exception as e:
        print(f"❌ エラー: {e}")
    
    return None

# ダウンロード実行
data_path = download_vesuvius_data()
print(f"\nデータパス: {data_path}" if data_path else "⚠️ データダウンロードが必要です")

## 🔧 Step 2: 必要なライブラリインポート

In [ ]:
# 必要なライブラリ
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List
from pathlib import Path

# PIL/cv2のインストール確認
try:
    from PIL import Image
    import cv2
    print("✅ 画像処理ライブラリ準備完了")
except ImportError:
    print("📦 画像処理ライブラリインストール中...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pillow", "opencv-python", "-q"])
    from PIL import Image
    import cv2
    print("✅ インストール完了")

# GPU確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n🖥️ 使用デバイス: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 📊 Step 3: データセットクラス（実データ対応）

In [ ]:
class VesuviusDataset(Dataset):
    """Vesuvius実データセット"""
    
    def __init__(self, split='train', volume_size=(96, 96, 32), num_samples=10):
        self.split = split
        self.volume_size = volume_size
        self.num_samples = num_samples
        
        # データパス検出
        self.data_path = self._find_data()
        
        # データロード
        self.volumes, self.labels = self._load_data()
    
    def _find_data(self):
        """データパス自動検出"""
        paths = [
            "/workspace/data/vesuvius-challenge-surface-detection",
            "./data/vesuvius-challenge-surface-detection",
            "/workspace/vesuvius-challenge-surface-detection",
            "./vesuvius-challenge-surface-detection",
            "/content/vesuvius-challenge-surface-detection"
        ]
        
        for path in paths:
            p = Path(path)
            if p.exists():
                train_images = p / "train_images"
                if train_images.exists():
                    tiffs = list(train_images.glob("*.tif"))
                    if len(tiffs) > 0:
                        print(f"✅ 実データ発見: {p} ({len(tiffs)}画像)")
                        return str(p)
        
        print("📊 実データなし - デモデータ使用")
        return None
    
    def _load_data(self):
        """データロード"""
        if self.data_path:
            return self._load_real_data()
        else:
            return self._generate_demo_data()
    
    def _load_real_data(self):
        """実データロード"""
        volumes = []
        labels = []
        
        data_path = Path(self.data_path)
        train_images_dir = data_path / "train_images"
        train_labels_dir = data_path / "train_labels"
        
        tiff_files = sorted(list(train_images_dir.glob("*.tif")))
        
        if len(tiff_files) == 0:
            return self._generate_demo_data()
        
        print(f"📂 {len(tiff_files)}個のTIFFファイルから{self.num_samples}個のボリューム作成")
        
        H, W, D = self.volume_size
        
        for vol_idx in range(min(self.num_samples, max(1, len(tiff_files) // D))):
            # スライス選択
            start_idx = (vol_idx * D) % max(1, len(tiff_files) - D)
            selected_files = tiff_files[start_idx:start_idx + D]
            
            if len(selected_files) < D:
                selected_files = selected_files + tiff_files[:D-len(selected_files)]
            
            volume_slices = []
            label_slices = []
            
            for tiff_file in selected_files[:D]:
                try:
                    # 画像読み込み
                    img = np.array(Image.open(tiff_file), dtype=np.float32)
                    
                    # グレースケール化
                    if len(img.shape) == 3:
                        img = img.mean(axis=2)
                    
                    # リサイズ
                    img = cv2.resize(img, (W, H))
                    
                    # 正規化
                    img = (img - img.mean()) / (img.std() + 1e-8)
                    
                    # ラベル処理
                    if train_labels_dir.exists():
                        label_file = train_labels_dir / tiff_file.name
                        if label_file.exists():
                            label = np.array(Image.open(label_file), dtype=np.uint8)
                            if len(label.shape) == 3:
                                label = label.mean(axis=2)
                            label = cv2.resize(label, (W, H), interpolation=cv2.INTER_NEAREST)
                            label = (label > 127).astype(np.int64)
                        else:
                            label = (img > np.percentile(img, 75)).astype(np.int64)
                    else:
                        label = (img > np.percentile(img, 75)).astype(np.int64)
                    
                    volume_slices.append(img)
                    label_slices.append(label)
                    
                except Exception as e:
                    # エラー時はランダムスライス
                    volume_slices.append(np.random.randn(H, W).astype(np.float32))
                    label_slices.append(np.zeros((H, W), dtype=np.int64))
            
            # 3Dボリューム構築
            volume = np.stack(volume_slices, axis=2)
            label = np.stack(label_slices, axis=2)
            
            volumes.append(volume)
            labels.append(label)
            
            if vol_idx == 0:
                print(f"  サンプルボリューム: {volume.shape}, 前景率: {(label==1).mean():.2%}")
        
        print(f"✅ {len(volumes)}個の実データボリューム作成完了")
        return volumes, labels
    
    def _generate_demo_data(self):
        """デモデータ生成"""
        volumes = []
        labels = []
        
        H, W, D = self.volume_size
        
        for i in range(self.num_samples):
            # リアルなボリューム生成
            volume = np.random.randn(H, W, D).astype(np.float32)
            
            # テクスチャ追加
            for z in range(D):
                x, y = np.meshgrid(np.linspace(0, 3*np.pi, H), np.linspace(0, 3*np.pi, W))
                pattern = np.sin(x + i) * np.cos(y + z/10)
                volume[:, :, z] += pattern * 0.5
                volume[:, :, z] += np.random.randn(H, W) * 0.2
            
            # 正規化
            volume = (volume - volume.mean()) / (volume.std() + 1e-8)
            
            # ラベル生成
            label = np.zeros((H, W, D), dtype=np.int64)
            num_regions = np.random.randint(3, 8)
            
            for _ in range(num_regions):
                cx, cy, cz = np.random.randint(H//4, 3*H//4), np.random.randint(W//4, 3*W//4), np.random.randint(D//4, 3*D//4)
                size = np.random.randint(5, 15)
                
                for x in range(max(0, cx-size), min(H, cx+size)):
                    for y in range(max(0, cy-size), min(W, cy+size)):
                        for z in range(max(0, cz-size//2), min(D, cz+size//2)):
                            if ((x-cx)**2 + (y-cy)**2 + (z-cz)**2*4) / size**2 < 1:
                                label[x, y, z] = 1
            
            volumes.append(volume)
            labels.append(label)
        
        return volumes, labels
    
    def __len__(self):
        return len(self.volumes)
    
    def __getitem__(self, idx):
        volume = torch.FloatTensor(self.volumes[idx]).unsqueeze(0)
        label = torch.LongTensor(self.labels[idx])
        return {'data': volume, 'target': label}

## 🏗️ Step 4: 3D U-Netモデル

In [ ]:
class UNet3D(nn.Module):
    """3D U-Net"""
    
    def __init__(self, in_channels=1, out_channels=2, init_features=32):
        super(UNet3D, self).__init__()
        
        features = init_features
        
        # Encoder
        self.encoder1 = self._block(in_channels, features)
        self.pool1 = nn.MaxPool3d(2, 2)
        
        self.encoder2 = self._block(features, features * 2)
        self.pool2 = nn.MaxPool3d(2, 2)
        
        self.encoder3 = self._block(features * 2, features * 4)
        self.pool3 = nn.MaxPool3d(2, 2)
        
        self.encoder4 = self._block(features * 4, features * 8)
        self.pool4 = nn.MaxPool3d(2, 2)
        
        # Bottleneck
        self.bottleneck = self._block(features * 8, features * 16)
        
        # Decoder
        self.upconv4 = nn.ConvTranspose3d(features * 16, features * 8, 2, 2)
        self.decoder4 = self._block((features * 8) * 2, features * 8)
        
        self.upconv3 = nn.ConvTranspose3d(features * 8, features * 4, 2, 2)
        self.decoder3 = self._block((features * 4) * 2, features * 4)
        
        self.upconv2 = nn.ConvTranspose3d(features * 4, features * 2, 2, 2)
        self.decoder2 = self._block((features * 2) * 2, features * 2)
        
        self.upconv1 = nn.ConvTranspose3d(features * 2, features, 2, 2)
        self.decoder1 = self._block(features * 2, features)
        
        self.conv = nn.Conv3d(features, out_channels, 1)
        
    def _block(self, in_channels, features):
        return nn.Sequential(
            nn.Conv3d(in_channels, features, 3, padding=1, bias=False),
            nn.BatchNorm3d(features),
            nn.ReLU(inplace=True),
            nn.Conv3d(features, features, 3, padding=1, bias=False),
            nn.BatchNorm3d(features),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))
        
        bottleneck = self.bottleneck(self.pool4(enc4))
        
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        
        return self.conv(dec1)

print("✅ モデル定義完了")

## 📊 Step 5: データローダー作成

In [ ]:
# データローダー作成
print("📊 データローダー作成中...\n")

# データセット作成
train_dataset = VesuviusDataset(
    split='train',
    volume_size=(96, 96, 32),  # GPU制限に応じて調整
    num_samples=20  # 訓練サンプル数
)

val_dataset = VesuviusDataset(
    split='val',
    volume_size=(96, 96, 32),
    num_samples=5  # 検証サンプル数
)

# データローダー
train_loader = DataLoader(
    train_dataset,
    batch_size=2,  # GPU制限に応じて調整
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

print(f"✅ データローダー作成完了")
print(f"   訓練: {len(train_dataset)}サンプル / {len(train_loader)}バッチ")
print(f"   検証: {len(val_dataset)}サンプル / {len(val_loader)}バッチ")

# サンプル確認
sample = next(iter(train_loader))
print(f"\n📐 データ形状:")
print(f"   入力: {sample['data'].shape}")
print(f"   ラベル: {sample['target'].shape}")

## ⚙️ Step 6: 学習設定

In [ ]:
# モデル初期化
model = UNet3D(in_channels=1, out_channels=2, init_features=32).to(device)

# 損失関数とオプティマイザー
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# パラメータ数
total_params = sum(p.numel() for p in model.parameters())
print(f"📊 モデルパラメータ数: {total_params:,}")
print(f"⚙️ 学習設定完了")

## 🏋️ Step 7: 学習実行

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(loader, desc="Training"):
        data = batch['data'].to(device)
        target = batch['target'].to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            data = batch['data'].to(device)
            target = batch['target'].to(device)
            
            output = model(data)
            loss = criterion(output, target)
            
            total_loss += loss.item()
    
    return total_loss / len(loader)

In [ ]:
# 学習ループ
num_epochs = 20
train_losses = []
val_losses = []
best_val_loss = float('inf')

print(f"🎓 学習開始 ({num_epochs}エポック)\n")

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print("-" * 50)
    
    # 学習
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # 検証
    val_loss = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # スケジューラー
    scheduler.step(val_loss)
    
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # ベストモデル保存
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, 'best_model.pth')
        print("✅ ベストモデル保存")

print(f"\n🎉 学習完了!")
print(f"Best Val Loss: {best_val_loss:.4f}")

## 📈 Step 8: 結果可視化

In [ ]:
# 学習曲線
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', linewidth=2)
plt.plot(val_losses, label='Val Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
last_n = min(10, len(train_losses))
plt.plot(train_losses[-last_n:], label='Train Loss', linewidth=2)
plt.plot(val_losses[-last_n:], label='Val Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'Last {last_n} Epochs')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Step 9: 予測サンプル確認

In [ ]:
# 予測実行
model.eval()
sample_batch = next(iter(val_loader))
data = sample_batch['data'].to(device)
target = sample_batch['target'].to(device)

with torch.no_grad():
    output = model(data)
    pred = output.argmax(dim=1)

# 可視化
data_cpu = data.cpu().numpy()
target_cpu = target.cpu().numpy()
pred_cpu = pred.cpu().numpy()

idx = 0
slice_idx = data_cpu.shape[-1] // 2

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(data_cpu[idx, 0, :, :, slice_idx], cmap='gray')
plt.title('Input')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(target_cpu[idx, :, :, slice_idx], cmap='tab20')
plt.title('Ground Truth')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(pred_cpu[idx, :, :, slice_idx], cmap='tab20')
plt.title('Prediction')
plt.axis('off')

plt.suptitle(f'Sample Prediction (Slice {slice_idx})')
plt.tight_layout()
plt.show()

# Dice係数
intersection = ((pred_cpu == 1) & (target_cpu == 1)).sum()
union = (pred_cpu == 1).sum() + (target_cpu == 1).sum()
dice = 2.0 * intersection / (union + 1e-8)
print(f"\nDice係数: {dice:.4f}")

## 💾 Step 10: モデル保存

In [ ]:
# 最終モデル保存
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'config': {
        'volume_size': (96, 96, 32),
        'batch_size': 2,
        'num_epochs': num_epochs,
        'best_val_loss': best_val_loss,
        'data_type': 'real' if train_dataset.data_path else 'demo'
    }
}, 'final_model.pth')

print("✅ モデル保存完了")
print("   - best_model.pth (ベストモデル)")
print("   - final_model.pth (最終モデル)")
print(f"\n📊 最終結果:")
print(f"   データタイプ: {'実データ' if train_dataset.data_path else 'デモデータ'}")
print(f"   最終Train Loss: {train_losses[-1]:.4f}")
print(f"   最終Val Loss: {val_losses[-1]:.4f}")
print(f"   ベストVal Loss: {best_val_loss:.4f}")

## 🎯 完了！

**Vesuvius Challenge 3D Surface Detection学習完了**

### ✅ このノートブックの特徴:
- **完全独立** - 外部ファイル不要
- **実データ対応** - Kaggleデータ自動ダウンロード
- **Runpods対応** - workspace環境自動検出
- **フォールバック** - 実データなしでもデモで動作

### 📂 保存ファイル:
- `best_model.pth` - 最良検証損失のモデル
- `final_model.pth` - 最終エポックのモデル

### 🚀 次のステップ:
1. より大きなボリュームサイズで再学習
2. エポック数を増やして精度向上
3. テストデータで推論実行
4. Kaggle提出ファイル作成